In [1]:
!pip install bitmex
!pip install python-binance
!pip install keras_multi_head
!pip install requests --upgrade
!pip install sktime
!pip install keras-self-attention

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67672 sha256=e2b4a05512a0a72348a7f4c4c1239634a1900990c5543f909b34fd3ae23aec97
  Stored in directory: /root/.cache/pip/wheels/42/35/4a/44ec4c358db21a5d63ed4e40f0f0012a438106f220bce4ccba
Successfully built bravado-core
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:00
  Prepari

In [2]:
# IMPORTS
import requests
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, QuantileTransformer, StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import tensorflow as tf
from keras_multi_head import MultiHeadAttention
import pandas
from pandas_datareader import data as pdr
import yfinance as yfin
from keras import backend as K
from sktime.transformations.series.boxcox import LogTransformer
from keras_self_attention import SeqSelfAttention

### API
bitmex_api_key = 'QJztZ3vF61eSlyYdDe1xnCrL'    #Enter your own API-key here
bitmex_api_secret = 'iV05s1WvMdagCcerBaW-OglyS5DR41KEM7ti148vG67-lwA7' #Enter your own API-secret here
binance_api_key = 'buEUMq3Qo5LaTwCOKl4vFJG1fe9kCmJtOtXOaPlpr7wgtF3iwXiVC3yKPFLTviyO'    #Enter your own API-key here
binance_api_secret = 'lpHoAXrM2n42H9W2cOlnQUD1zckp1v6PiaCjQuLdgoPrCJ4jaBA8NGHxDd4OodSA' #Enter your own API-secret here

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 1000
bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
#binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new


def get_crypto_price(symbol, exchange, start_date = None):
    api_key = 'RC2YY8JWEUDMY6IW'
    api_url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={exchange}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    print(raw_df)
    df = pd.DataFrame(raw_df['Time Series (Digital Currency Daily)']).T
    df = df.rename(columns = {'1a. open (USD)': 'open', '2a. high (USD)': 'high', '3a. low (USD)': 'low', '4a. close (USD)': 'close', '5. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1].drop(['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', '4b. close (USD)', '6. market cap (USD)'], axis = 1)
    if start_date:
        df = df[df.index >= start_date]
    return df


#def get_all_binance(symbol, kline_size, save = False):
#    filename = '%s-%s-data.csv' % (symbol, kline_size)
#    if os.path.isfile(filename): data_df = pd.read_csv(filename)
#    else: data_df = pd.DataFrame()
#    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
#    delta_min = (newest_point - oldest_point).total_seconds()/60
#    available_data = math.ceil(delta_min/binsizes[kline_size])
#    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
#    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
#    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
#    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
#    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
#    if len(data_df) > 0:
#        temp_df = pd.DataFrame(data)
#        data_df = data_df.append(temp_df)
#    else: data_df = data
#    data_df.set_index('timestamp', inplace=True)
#    if save: data_df.to_csv(filename)
#    print('All caught up..!')
#    return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

/usr/local/lib/python3.10/dist-packages/swagger_spec_validator/validator20.py:49: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(


In [3]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [4]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('loss') <= 1e-3:
            self.model.stop_training = True

In [5]:
def getFuturePrices(stock, exchange, dict_output, lookback, startdate, enddate):
  symbol = stock #'MATICUSDT'
  #data = get_all_binance(symbol, '1d', save = False)
  #data['timestamp'] = data.index
  #data['timestamp'] = data['timestamp'].astype(str).str[:10]
  crypto = stock + '-' + exchange
  yfin.pdr_override()
  data = pdr.get_data_yahoo(crypto, start=startdate, end=enddate)
  df = data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
  df.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
  eps = 1e-9
  df['Open'] = df['Open'].astype('float32') + eps
  df['High'] = df['High'].astype('float32') + eps
  df['Low'] = df['Low'].astype('float32') + eps
  df['Close'] = df['Close'].astype('float32') + eps
  df['Adj Close'] = df['Adj Close'].astype('float32') + eps
  df['Volume'] = df['Volume'].astype('float32') + eps
  #df = df.drop("Date", axis=1)
  print(df)

  if(len(df) < lookback):
    return(dict_output)

  train = df[:(-1*lookback)-1]
  scalers={}
  for i in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']:
      scaler = pipe = Pipeline([('log', LogTransformer()), ('scaler', StandardScaler()), ('max', MinMaxScaler())]) #MinMaxScaler()
      s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
      s_s=np.reshape(s_s,len(s_s))
      scalers['scaler_'+ i] = scaler
      train[i]=s_s
  test = df[(-1*lookback)-1:]
  for i in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']:
      scaler = scalers['scaler_'+i]
      s_s = scaler.transform(test[i].values.reshape(-1,1))
      s_s=np.reshape(s_s,len(s_s))
      scalers['scaler_'+i] = scaler
      test[i]=s_s

  n_past = lookback
  n_future = 1
  n_features = 6

  X_train, y_train = split_series(train.values, n_past, n_future)
  X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
  y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
  X_test, y_test = split_series(test.values,n_past, n_future)
  X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
  y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

  X_train = np.asarray(X_train).astype('float32')
  y_train = np.asarray(y_train).astype('float32')
  X_test = np.asarray(X_test).astype('float32')
  y_test = np.asarray(y_test).astype('float32')

  # E1D1
  # n_features ==> no of features at each timestep in the data.
  #
  encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
  encoder_l1 = tf.keras.layers.LSTM(1500, dropout=0.2, recurrent_dropout=0.2, return_state=True)
  encoder_outputs1 = encoder_l1(encoder_inputs)

  encoder_states1 = encoder_outputs1[1:]

  #
  decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

  #
  decoder_l1 = tf.keras.layers.LSTM(1500, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
  decoder_l2 = SeqSelfAttention(
    attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
    attention_activation='tanh',
    name='Attention',
  )(decoder_l1)
  decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features, activation='linear'))(decoder_l2)

  #
  model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

  #
  model_e1d1.summary()

  callback = CustomCallback()

  reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
  model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
  history_e1d1=model_e1d1.fit(X_train,y_train,epochs=200,validation_data=(X_test,y_test),batch_size=4096,callbacks=[callback])

  test2 = X_test
  final_values = np.zeros(shape=(120,6))
  for i in range(120):
    pred_e1d1=model_e1d1.predict(test2)
    for j in range(lookback):
      if j != (lookback-1):
        test2[0][j] = test2[0][j+1]
      else:
        test2[0][j] = pred_e1d1[0]

    final_values[i] = pred_e1d1[0]

  #test2 = np.array([test2])
  final_values = np.array([final_values])

  pred_e1d1 = final_values
  for index,i in enumerate(df.columns):
      scaler = scalers['scaler_'+i]
      pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
      y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
      y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

  final_df = pd.DataFrame(pred_e1d1[0])
  final_df.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
  print(final_df)
  dict_output[crypto] = (df['Adj Close'].iloc[-1], final_df['Adj Close'].max())

  K.clear_session()
  del model_e1d1

  print(dict_output)
  return(dict_output)

In [6]:
#crypto_df = pd.read_csv('cryptos.csv')
#cryptos = crypto_df['currency code'].tolist()

In [7]:
import requests

cryptos = []
url = 'https://web-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

for start in range(1, 20000, 5000):

    params = {
        'start': start,
        'limit': 5000,
    }

    r = requests.get(url, params=params)
    data = r.json()

    for number, item in enumerate(data['data']):
        print(f"{start+number:4} | {item['symbol']:5} | {item['date_added'][:10]}")
        cryptos.append(item['symbol'])


Streaming output truncated to the last 5000 lines.
5040 | TIDE  | 2023-09-02
5041 | OTSEA | 2023-11-03
5042 | KUMA  | 2021-06-10
5043 | SPARKO | 2023-11-14
5044 | BIIS  | 2023-12-19
5045 | LOVE  | 2023-05-30
5046 | AXLW  | 2024-05-08
5047 | LOON  | 2021-01-04
5048 | RICE  | 2021-10-16
5049 | ASTRO | 2023-02-06
5050 | ESW   | 2021-08-03
5051 | CYS   | 2021-06-02
5052 | ZEUM  | 2022-02-08
5053 | HYCO  | 2022-11-18
5054 | TONK  | 2024-03-27
5055 | QASH  | 2017-11-21
5056 | CASINU | 2024-02-05
5057 | DEFIDO | 2021-10-08
5058 | PALM  | 2022-10-28
5059 | FMC   | 2023-12-25
5060 | MONKE | 2024-05-09
5061 | MMY   | 2022-12-21
5062 | STZ   | 2021-12-12
5063 | CEM   | 2023-05-18
5064 | MFERS | 2024-03-19
5065 | ATEC  | 2024-04-15
5066 | BRS   | 2022-04-20
5067 | DEFX  | 2021-05-29
5068 | MILEI | 2024-01-07
5069 | CS    | 2018-03-02
5070 | STATS | 2024-03-27
5071 | PROB  | 2019-09-13
5072 | SINGLE | 2022-01-19
5073 | HSC   | 2018-07-09
5074 | MDA   | 2017-09-11
5075 | STELLA | 2022-01-16
5076 | P

In [8]:
import random

dict_output = {}

for i in range(100):
  n = random.randint(0,len(cryptos))
  for j in ['BTC', 'ETH', 'USD']:
    #n = i
    print(cryptos[n])
    dict_output = getFuturePrices(cryptos[n], j, dict_output, 20, '2011-01-01', '2024-05-23')

ZKML
yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZKML-BTC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
ZKML


[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZKML-ETH']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
<ipython-input-5-8df38debae25>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
ZKML
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2024-02-19  0.162845  0.183311  0.126104  0.143851   0.143851  1866466.0
2024-02-20  0.144550  0.185782  0.122543  0.185782   0.185782  1403724.0
2024-02-21  0.185782  0.233969  0.145647  0.212327   0.212327  2807982.0
2024-02-22  0.212327  0.225810  0.166749  0.180452   0.180452  1510409.0
2024-02-23  0.180452  0.224887  0.163448  0.216972   0.216972  1517530.0
...              ...       ...       ...       ...        ...        ...
2024-05-18  0.163053  0.180593  0.149044  0.149776   0.149776   143878.0
2024-05-19  0.149776  0.151487  0.125920  0.133942   0.133942   117615.0
2024-05-20  0.133942  0.209735  0.133801  0.209735   0.209735   250229.0
2024-05-21  0.209735  0.217984  0.189355  0.199079   0.199079   320631.0
2024-05-22  0.199079  0.199860  0.152809

<ipython-input-5-8df38debae25>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-5-8df38debae25>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-5-8df38debae25>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20, 6)]              0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 1500),               9042000   ['input_1[0][0]']             
                              (None, 1500),                                                       
                              (None, 1500)]                                                       
                                                                                                  
 repeat_vector (RepeatVecto  (None, 1, 1500)              0         ['lstm[0][0]']                
 r)                                                                                           

KeyboardInterrupt: 

In [ ]:
dict_output

In [ ]:
dict_output

In [11]:
dict_output = {}
dict_output = getFuturePrices('WOLF', 'USD', dict_output, 60, '2011-01-01', '2024-05-23')

[*********************100%%**********************]  1 of 1 completed
<ipython-input-5-8df38debae25>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-5-8df38debae25>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-5-8df38debae25>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

                    Open          High           Low         Close  \
Date                                                                 
2021-04-07  1.000100e-05  1.100100e-05  1.000100e-05  1.100100e-05   
2021-04-08  1.100100e-05  1.600100e-05  1.000100e-05  1.600100e-05   
2021-04-09  1.600100e-05  1.900100e-05  1.600100e-05  1.700100e-05   
2021-04-10  1.700100e-05  2.400100e-05  1.700100e-05  2.100100e-05   
2021-04-11  2.100100e-05  2.400100e-05  2.100100e-05  2.200100e-05   
...                  ...           ...           ...           ...   
2024-05-18  1.000000e-09  1.000000e-09  1.000000e-09  1.000000e-09   
2024-05-19  1.000000e-09  1.000000e-09  1.000000e-09  1.000000e-09   
2024-05-20  1.000000e-09  1.000000e-09  1.000000e-09  1.000000e-09   
2024-05-21  1.000000e-09  1.000000e-09  1.000000e-09  1.000000e-09   
2024-05-22  1.000000e-09  1.000000e-09  1.000000e-09  1.000000e-09   

               Adj Close        Volume  
Date                                    
2021-04

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 60, 6)]              0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 1500),               9042000   ['input_1[0][0]']             
                              (None, 1500),                                                       
                              (None, 1500)]                                                       
                                                                                                  
 repeat_vector (RepeatVecto  (None, 1, 1500)              0         ['lstm[0][0]']                
 r)                                                                                           